Imports necessary libraries: numpy for numerical operations, cv2 for OpenCV functions, sys for system-specific parameters and functions

In [43]:
import numpy as np                    #complete this
import sys                    #complete this
import cv2  

Defines colors for text and border using random RGB values, sets a font for text, and specifies the path to the video source.

In [44]:
TEXT_COLOR = (0, 255, 0)
TRACKER_COLOR = (255, 0, 0)
FONT = cv2.FONT_HERSHEY_SIMPLEX

VIDEO_SOURCE = "Animal_1.mp4"             #complete this

Lists the types of background subtraction methods available. reference: https://stackoverflow.com/questions/33266239/differences-between-mog-mog2-and-gmg-in-opencv-cv2

In [45]:
BGS_TYPES = ["GMG", "MOG", "MOG2", "KNN", "CNT"]             #complete this
BGS_TYPE = BGS_TYPES[2]       

Defines a function to get different types of kernels (structuring elements) used in morphological transformations.

In [46]:
def getKernel(KERNEL_TYPE):
    if KERNEL_TYPE == "dilation":
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    if KERNEL_TYPE == "opening":
        kernel = np.ones((3, 3), np.uint8)
    if KERNEL_TYPE == "closing":
        kernel = np.ones((3, 3), np.uint8)

    return kernel

Applies specified morphological operations to an image, like closing, opening, or dilation, and returns the modified image.

In [47]:
def getFilter(img, filter):
    if filter == 'closing':
        return cv2.morphologyEx(img, cv2.MORPH_CLOSE, getKernel("closing"), iterations=1)

    if filter == 'opening':
        return cv2.morphologyEx(img, cv2.MORPH_OPEN, getKernel("opening"), iterations=1)

    if filter == 'dilation':
        return cv2.dilate(img, getKernel("dilation"), iterations=2)

    if filter == 'combine':
        closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, getKernel("closing"), iterations=1)
        opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, getKernel("opening"), iterations=1)
        dilation = cv2.dilate(opening, getKernel("dilation"), iterations=1)

        return dilation

Based on the type (BGS_TYPE), it initializes and returns a background subtractor object. Each type corresponds to a different algorithm for background subtraction.

In [48]:
def getBGSubtractor(BGS_TYPE):
    if BGS_TYPE == "GMG":                                     #complete this
        return cv2.bgsegm.createBackgroundSubtractorGMG()
    if BGS_TYPE == "MOG":                                     #complete this
        return cv2.bgsegm.createBackgroundSubtractorMOG()
    if BGS_TYPE == "MOG2":                                     #complete this
        return cv2.createBackgroundSubtractorMOG2()
    if BGS_TYPE == "KNN":                                     #complete this
        return cv2.createBackgroundSubtractorKNN()
    if BGS_TYPE == "CNT":                                     #complete this
        return cv2.bgsegm.createBackgroundSubtractorCNT()
    print("Invalid detector")
    sys.exit(1)

Prepare the video for processing, initialize the background subtractor, and set a minimum area for detected objects.

In [49]:
cap = cv2.VideoCapture(VIDEO_SOURCE)                                            #complete this
bg_subtractor = getBGSubtractor(BGS_TYPE)
minArea = 250                                       #complete this

This function contains the main loop for processing the video:

1.Read and Resize Frame: Each frame of the video is read and resized for processing efficiency.<br>
2.Apply Background Subtraction: The selected background subtraction method is applied to separate moving objects from the background.<br>
3.Morphological Filtering: The background mask is cleaned up using a combination of morphological operations to improve the detection of moving objects.<br>
4.Find Contours: Contours of moving objects are detected in the filtered mask.<br>
5.Drawing and Display: For each detected object with an area larger than minArea, the script dynamically draws a semi-transparent overlay on the original frame to highlight the object.<br>
6.Display Results: The processed frame and the mask are displayed in separate windows.<br>
7.Exit Condition: The loop can be exited by pressing the "q" key.

In [50]:
def main():
    while (cap.isOpened):
        ok, frame = cap.read()                              
        if not ok:
            print("Finished processing the video")
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            break

        frame = cv2.resize(frame, (0, 0), fx=0.50, fy=0.50)

        bg_mask = bg_subtractor.apply(frame)
        bg_mask = getFilter(bg_mask, 'combine')
        bg_mask = cv2.medianBlur(bg_mask, 5)

        (contours, hierarchy) = cv2.findContours(bg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        #print(contours)
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area >= minArea:
                x, y, w, h = cv2.boundingRect(cnt)
                cv2.rectangle(frame, (10,30), (250,55), (255,0,0), -1)
                cv2.putText(frame, 'Motion Detected', (10,50), FONT, 0.8, TEXT_COLOR, 2, cv2.LINE_AA)             #put any alert msg e.g. Motion detected!

                #Rectangle
                cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 0, 0), 3)

                #Countour
                #cv2.drawContours(frame, cnt, -1, (0,0,255), 3)

                #https://www.pyimagesearch.com/2016/03/07/transparent-overlays-with-opencv/
                # for alpha in np.arange(0.8, 1.1, 0.9)[::-1]:
                #     frame_copy = frame.copy()
                #     output = frame.copy()
                #     cv2.drawContours(frame_copy, [cnt], -1, TRACKER_COLOR, -1)
                #     frame = cv2.addWeighted(frame_copy, alpha, output, 1 - alpha, 0, output)


        result = cv2.bitwise_and(frame, frame, mask=bg_mask)
        cv2.imshow('Frame', frame)
        cv2.imshow('Mask', result)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
            break

main()